# Where to Open a Coffee Shop in Atlanta, GA

## Introduction

_Opening a coffee shop, or any restaurant for that matter, can be extremely difficult - especially during the current pandemic. One of the most difficult and critical choices of any business owner is locating their shop. In the restaurant business a good location is often a significant factor in the ability of the restaurant to make it and suceed. The goal of this project and analysis is to determine what locations within the city of Atlanta potentially offer a business owner the ideal location for a coffee shop targeting working adults_

This "ideal" location will be defined by a few critical factors: 
* Proximity to other coffee shops
* Proximity to other restaurants 
* Proximity to offices to target working adults

## Data Sources

In order to complete this analysis some critical data needs to be located:
* Location of restaurants in Atlanta
* Location of coffee shops in Atlanta
* Location of offices in Atlanta

To get this data the follow sources will need to be used:

* Foursquare API to help provide the locations of various restaurants including their type (i.e. coffee shop)
* Foursquare API to help provide the locations of various offices 
* List of neighborhoods in Atlanta which has been made available by the Atlanta Regianal Commision https://opendata.atlantaregional.com/ 

This data will allow us to compare the various parts of Atlanta to determine which neighborhood will be an ideal cadidate for a coffee shop targeting working adults